# Drag coefficient

In [ ]:
import sys

if '..' not in sys.path:
    sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib import colors

import helper
import seaborn

plt.style.use([helper.araa_style])

In [ ]:
Mach = 1e-3

In [ ]:
Re = np.logspace(-4, 5, 300)
Cd = np.zeros_like(Re)

Here we define `M = vrel / vth` (this is not quite exact as it should be cs). Actual Mach number is $M= v / c _ s$ and $v _ {th} = \sqrt{\frac{8}{\pi}}\, c _ s$, so

$$M _ {here} = M _ {actual} \, \sqrt{\frac{\pi}{8}}$$

In [ ]:
M = Mach * np.sqrt(np.pi / 8)

We derive the Knudsen number from the given Reynolds grid and "Mach" number

In [ ]:
Kn = 4 * M / Re

This is the classic Weidenschilling case

In [ ]:
mask_E = (Kn>4/9)
Cd_E = 8./(3. * M)  # Epstein drag
Cd[mask_E] = Cd_E  # Epstein drag

# first Stokes regime
mask_St1 = (Kn<4/9) & (Re<=1.0)
Cd[mask_St1] = 24. / Re[mask_St1]

# second Stokes regime
mask_St2 = (Re > 1) & (Re < 800.)
Cd[mask_St2] = 24. * Re[mask_St2]**-0.6

# Newton drag
mask_N = Re >= 800.
Cd[mask_N] = 0.44

This is from [Cheng 2009](https://doi.org/10.1016/j.powtec.2008.07.006).

In [ ]:
def Cd_cheng(Re, M):
    
    Cd = 24. / Re * (1 + 0.27 * Re)**0.43 + 0.47 * (1.0 - np.exp(-0.04 * Re**0.38))
    Cd_E = 8./(3. * M)  # Epstein drag
    
    return np.minimum(Cd_E, Cd)

In [ ]:
texts = [
    r'$\mathsf{Kn} \geq \frac{4}{9}$',
    r'$\mathsf{Kn} < \frac{4}{9}, \mathsf{Re} < 1$',
    r'$1 \leq \mathsf{Re} < 800$',
    r'$\mathsf{Re} \geq 800$',
    ]

In [ ]:
fact = 2

f, ax = plt.subplots(dpi=200)

ax.loglog(Re, Cd, 'k--', label='Weidenschilling 1977')
ax.loglog(Re, Cd_cheng(Re, M), label=f'Cheng 2009 ($M = {{{Mach:.3f}}}$)')

# semi-transparent line
col = colors.to_rgb('C2') + (0.0,)
mycm = colors.LinearSegmentedColormap.from_list('my',[col, 'C2'])
points = np.vstack((Re, Cd_cheng(Re, fact * M))).T.reshape(-1, 1, 2)
segments = np.hstack((points[:-1], points[1:]))
alphas = np.maximum(1 - (np.log10(Re) + 4) / 4, 0)
lc = LineCollection(segments, array=alphas, cmap=mycm,
                    lw=1.5, zorder=500)
line = ax.add_collection(lc)
ax.plot([], [], c='C2', label=f'Cheng 2009 ($M = {{{fact * Mach:.3f}}}$)')

ylim = [1e-1, 5e3]


for mask, txt in zip([mask_E, mask_St1, mask_St2, mask_N], texts):
    idx = np.where(mask)[0][[0,-1]] + [0, 1]
    idx[1] = min(idx[1], len(Re) - 1)
    fill = ax.fill_between(Re[idx], ylim[0] * np.ones(2), ylim[1] * np.ones(2), alpha=0.25, zorder=-1)
    ax.text(np.sqrt(np.prod(Re[idx])), 2e-1, txt,
            fontsize='small', horizontalalignment='center',
            color=fill.get_facecolor()[0], alpha=1, zorder=10)

ax.set_xlim(Re[[0, -1]])
ax.set_ylim(ylim)
ax.set_xlabel('Reynolds number')
ax.set_ylabel('drag coefficient $C_d$')
ax.legend().get_frame().set_alpha(0.5);

In [ ]:
helper.apply_araa_style(ax)
f.savefig(helper.output_dir / 'drag_law.pdf', transparent=True)
display(f)